# 🔧 Feature Engineering & Preprocessing
## House Prices - Advanced Regression Techniques

**Objetivo:** Crear features nuevas, manejar missing values, encoding y transformaciones

**Pipeline:**
1. Manejo de valores faltantes
2. Remoción de outliers
3. Feature Engineering (crear 5+ features nuevas)
4. Encoding de variables categóricas
5. Transformación logarítmica del target
6. Escalado de features


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../src')

from preprocessing import HousePricePreprocessor
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Librerías importadas")


## 1. Carga de Datos


In [ ]:
train_raw = pd.read_csv('../data/train.csv')
test_raw = pd.read_csv('../data/test.csv')

print(f"Train shape: {train_raw.shape}")
print(f"Test shape: {test_raw.shape}")

y_train = train_raw['SalePrice'].copy()
train_features = train_raw.drop(columns=['SalePrice'])

print(f"\nTarget shape: {y_train.shape}")
print(f"Train features shape: {train_features.shape}")


## 2. Análisis de Missing Values


In [ ]:
def plot_missing_values(df, title='Missing Values'):
    missing = df.isnull().sum()
    missing_pct = 100 * missing / len(df)

    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing_Count': missing.values,
        'Missing_Pct': missing_pct.values
    })

    missing_df = missing_df[missing_df['Missing_Count'] > 0]\
                    .sort_values('Missing_Pct', ascending=False)

    if len(missing_df) > 0:
        fig, ax = plt.subplots(figsize=(10, 6))
        colors = plt.cm.Reds(np.linspace(0.4, 0.9, len(missing_df)))
        ax.barh(missing_df['Column'], missing_df['Missing_Pct'],
                color=colors, edgecolor='black')
        ax.set_title(title)
        plt.show()
    else:
        print("✅ No hay missing values")

    return missing_df

missing_before = plot_missing_values(
    train_features,
    'Missing Values - ANTES del Preprocessing'
)


## 3. Pipeline de Preprocessing


In [ ]:
preprocessor = HousePricePreprocessor()

X_train, y_train_log, X_test, test_ids = preprocessor.preprocess_pipeline(
    train_raw.copy(),
    test_raw.copy(),
    target_col='SalePrice'
)


In [ ]:
print("="*70)
print("VERIFICACIÓN DE MISSING VALUES DESPUÉS DEL PREPROCESSING")
print("="*70)

print(f"Train - Missing values: {X_train.isnull().sum().sum()}")
if X_test is not None:
    print(f"Test - Missing values: {X_test.isnull().sum().sum()}")

print("\n✅ Missing values manejados correctamente")


## 4. Análisis de Features Creadas


In [ ]:
new_features = [
    'TotalSF', 'TotalBath', 'HouseAge', 'RemodAge', 'WasRemodeled',
    'TotalQual', 'TotalPorchSF', 'HasPool', 'HasGarage',
    'HasBsmt', 'Has2ndFloor'
]

print("="*70)
print("FEATURES NUEVAS CREADAS")
print("="*70)

for f in new_features:
    print(f"✅ {f}" if f in X_train.columns else f"❌ {f}")

print(f"\n📊 Total de features finales: {X_train.shape[1]}")


In [ ]:
X_train_df = X_train if isinstance(X_train, pd.DataFrame) \
    else pd.DataFrame(X_train, columns=preprocessor.feature_names)

y_train_original = preprocessor.inverse_transform_target(y_train_log)

temp_df = X_train_df.copy()
temp_df['SalePrice'] = y_train_original

new_features_present = [f for f in new_features if f in temp_df.columns]

correlations_new = temp_df[new_features_present + ['SalePrice']]\
    .corr()['SalePrice']\
    .drop('SalePrice')\
    .sort_values(ascending=False)

print(correlations_new)


## 5. Análisis de la Transformación del Target


In [ ]:
from scipy.stats import skew, kurtosis

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(y_train_original, bins=50)
axes[0].set_title(f"Original\nSkew: {skew(y_train_original):.3f}")

axes[1].hist(y_train_log, bins=50, color='orange')
axes[1].set_title(f"Log\nSkew: {skew(y_train_log):.3f}")

plt.show()


## 6. Encoding de Variables Categóricas


In [ ]:
print(f"Features originales: 79")
print(f"Features finales: {X_train_df.shape[1]}")


## 7. Distribución de Features Numéricas


In [ ]:
numeric_cols = X_train_df.select_dtypes(include=[np.number]).columns
skewness = X_train_df[numeric_cols].apply(lambda x: skew(x.dropna()))
high_skew = skewness[abs(skewness) > 0.75]

print(high_skew.sort_values(ascending=False).head(15))


## 8. Guardar Datos Procesados


In [ ]:
np.save('../data/X_train_processed.npy', X_train_df.values)
np.save('../data/y_train_log.npy', y_train_log)

if X_test is not None:
    np.save('../data/X_test_processed.npy', X_test.values)
    np.save('../data/test_ids.npy', test_ids)

with open('../data/feature_names.txt', 'w') as f:
    for feat in preprocessor.feature_names:
        f.write(f"{feat}\n")

print("✅ Datos procesados guardados")
